In [1]:
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split 

In [2]:
FILE_PATH = 'fer2013.csv'
image_size = (48, 48)

In [3]:
data = pd.read_csv(FILE_PATH)   

In [4]:
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [5]:
pixels = data['pixels'].tolist()

In [6]:
width, height = image_size

In [7]:
# load images and emotions 
faces = []

for p in pixels:
    face = [int(pix) for pix in p.split(' ')]
    face = np.asarray(face).reshape(width, height)
    face = cv2.resize(face.astype('uint8'), image_size)
    faces.append(face.astype('float32'))

In [ ]:
len(faces)

In [8]:
faces = np.asarray(faces)
faces = np.expand_dims(faces, -1)

In [9]:
emotions = pd.get_dummies(data['emotion']).values
emotions.shape

(35887, 7)

In [10]:
# pre-process the images 
def preprocess(x, v2=True):  # v2 to keep the image btw. -1 and 1
    x = x.astype('float32')
    x = x/255.0
    if v2:
        x = (x - 0.5)*2.0
    return x

In [11]:
faces = preprocess(faces)

In [ ]:
# print('showing some sample training images')
# for image in np.arange(0,10):
#     cv2.namedWindow('some sample training images', cv2.WINDOW_NORMAL)
#     cv2.imshow('some sample training images',faces[image])
#     cv2.waitKey(500)
#     cv2.destroyAllWindows()

In [12]:
xtrain, xtest, ytrain, ytest = train_test_split(faces, emotions,test_size=0.2,shuffle=True)

In [13]:
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import layers

In [15]:
from keras.layers import Activation, Convolution2D, Conv2D, Dropout, AveragePooling2D, BatchNormalization, GlobalAveragePooling2D, Flatten, Input, MaxPooling2D, SeparableConv2D


In [16]:
from keras.models import Model
from keras.regularizers import l2

In [17]:
from tensorflow.keras.layers import DepthwiseConv2D, BatchNormalization, Activation

In [ ]:
# !pip install opencv-python

In [18]:
batch_size = 32
epochs = 100
image_shape = (48, 48, 1)
verbose = True 
num_class = 7
patience = 50  # number of epochs with no improvement after which training will be stopped
base_path = 'models/'
l2_regularization = 0.01

In [19]:
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

In [20]:
regularization = l2(l2_regularization)

In [21]:
# model
image_input = Input(image_shape)
x = Conv2D(filters=8, kernel_size=(3,3), strides=(1,1), kernel_regularizer=regularization, use_bias=False)(image_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(filters=8, kernel_size=(3,3), strides=(1,1), kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# module 1
# residual module 
residual = Conv2D(filters=16, kernel_size=(1,1), strides=(2,2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

# x = SeparableConv2D(filters=16, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = SeparableConv2D(filters=16, kernel_size=(3,3), padding='same', depthwise_regularizer=regularization, pointwise_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
# x = SeparableConv2D(filters=16, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = SeparableConv2D(filters=16, kernel_size=(3,3), padding='same', depthwise_regularizer=regularization, pointwise_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)
x = layers.add([x,residual])

# module 2
# residual module 
residual = Conv2D(filters=32, kernel_size=(1,1), strides=(2,2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

# x = SeparableConv2D(filters=32, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = SeparableConv2D(filters=32, kernel_size=(3,3), padding='same', depthwise_regularizer=regularization, pointwise_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
# x = SeparableConv2D(filters=32, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = SeparableConv2D(filters=32, kernel_size=(3,3), padding='same', depthwise_regularizer=regularization, pointwise_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)
x = layers.add([x,residual])

# module 3
# residual module 
residual = Conv2D(filters=64, kernel_size=(1,1), strides=(2,2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

# x = SeparableConv2D(filters=64, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = SeparableConv2D(filters=64, kernel_size=(3,3), padding='same', depthwise_regularizer=regularization, pointwise_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
# x = SeparableConv2D(filters=64, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = SeparableConv2D(filters=64, kernel_size=(3,3), padding='same', depthwise_regularizer=regularization, pointwise_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)
x = layers.add([x,residual])

# module 4
# residual module 
residual = Conv2D(filters=128, kernel_size=(1,1), strides=(2,2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

# x = SeparableConv2D(filters=128, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = SeparableConv2D(filters=128, kernel_size=(3,3), padding='same', depthwise_regularizer=regularization, pointwise_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
# x = SeparableConv2D(filters=128, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = SeparableConv2D(filters=128, kernel_size=(3,3), padding='same', depthwise_regularizer=regularization, pointwise_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)
x = layers.add([x,residual])

x = Conv2D(filters=num_class, kernel_size=(3,3), padding='same')(x)
x = GlobalAveragePooling2D()(x)

output = Activation('softmax', name='predictions')(x)

In [22]:
model = Model(image_input, output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

In [23]:
# callbacks 
log_file_path = base_path + '_emotion_training.log'
csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping(monitor='val_loss', patience=patience, verbose=1)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)

# trained_models_path = base_path + '_mini_xception'
# model_names = trained_models_path + '.{epoch:02d}_{val_acc:.2f}.hdf5'
# model_checkpoint = ModelCheckpoint(filepath=model_names, monitor='val_loss', verbose=verbose, save_best_only=True)

model_checkpoint = ModelCheckpoint('mini_xception.keras', monitor='val_acc', verbose=1, save_best_only=True)
csv_logger = CSVLogger('mini_xception_training.log')

In [24]:
callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]

model.fit(data_generator.flow(xtrain, ytrain, batch_size), 
          steps_per_epoch=len(xtrain)//batch_size, 
          epochs=epochs, 
          verbose=1, 
          callbacks=callbacks, 
          validation_data=(xtest, ytest))
model.save('mini_xception_model.h5')

Epoch 1/100
  3/897 ━━━━━━━━━━━━━━━━━━━━ 42s 47ms/step - accuracy: 0.2396 - loss: 6.4146

/Users/sana/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


897/897 ━━━━━━━━━━━━━━━━━━━━ 64s 68ms/step - accuracy: 0.2833 - loss: 4.1082 - val_accuracy: 0.4094 - val_loss: 1.9311 - learning_rate: 0.0010
Epoch 2/100
  1/897 ━━━━━━━━━━━━━━━━━━━━ 1:05 73ms/step - accuracy: 0.3125 - loss: 2.2676

/Users/sana/anaconda3/lib/python3.11/site-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_acc available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
2024-05-09 13:48:24.500031: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/sana/anaconda3/lib/python3.11/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


897/897 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.3125 - loss: 1.1351 - val_accuracy: 0.4106 - val_loss: 1.9274 - learning_rate: 0.0010
Epoch 3/100
897/897 ━━━━━━━━━━━━━━━━━━━━ 65s 72ms/step - accuracy: 0.4094 - loss: 1.8246 - val_accuracy: 0.4817 - val_loss: 1.5605 - learning_rate: 0.0010
Epoch 4/100
  1/897 ━━━━━━━━━━━━━━━━━━━━ 1:12 81ms/step - accuracy: 0.3438 - loss: 1.7028

...

897/897 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5625 - loss: 0.5375 - val_accuracy: 0.6000 - val_loss: 1.1340 - learning_rate: 1.0000e-06
Epoch 97/100
897/897 ━━━━━━━━━━━━━━━━━━━━ 61s 68ms/step - accuracy: 0.5940 - loss: 1.1599 - val_accuracy: 0.5992 - val_loss: 1.1345 - learning_rate: 1.0000e-06
Epoch 98/100
  1/897 ━━━━━━━━━━━━━━━━━━━━ 1:11 80ms/step - accuracy: 0.5625 - loss: 1.4633

2024-05-09 14:52:23.840485: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


897/897 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5625 - loss: 0.7325 - val_accuracy: 0.5991 - val_loss: 1.1346 - learning_rate: 1.0000e-06
Epoch 99/100
897/897 ━━━━━━━━━━━━━━━━━━━━ 60s 67ms/step - accuracy: 0.5940 - loss: 1.1618 - val_accuracy: 0.5991 - val_loss: 1.1340 - learning_rate: 1.0000e-06
Epoch 100/100
  1/897 ━━━━━━━━━━━━━━━━━━━━ 1:04 72ms/step - accuracy: 0.5312 - loss: 1.2600

2024-05-09 14:53:27.460592: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


897/897 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5312 - loss: 0.6307 - val_accuracy: 0.5991 - val_loss: 1.1340 - learning_rate: 1.0000e-06


In [25]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
tflite_model_path = 'mini_xception_model.tflite'
open(tflite_model_path, 'wb').write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/jd/0x5q_vls0_g80fd17j5cgvmw0000gn/T/tmpn575wcq6/assets


INFO:tensorflow:Assets written to: /var/folders/jd/0x5q_vls0_g80fd17j5cgvmw0000gn/T/tmpn575wcq6/assets


: 

In [2]:
import cv2
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [4]:
import numpy as np

# Load the input image
img = cv2.imread('dabke.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect faces in the image
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

In [ ]:
# Define the emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Iterate over the detected faces
for (x, y, w, h) in faces:
    # Extract the face region from the image
    face_roi = gray[y:y+h, x:x+w]
    
    # Preprocess the face region (e.g., resize, normalization)
    face_roi = cv2.resize(face_roi, (48, 48))
    face_roi = face_roi.astype('float32') / 255.0
    face_roi = np.expand_dims(face_roi, axis=0)
    face_roi = np.expand_dims(face_roi, axis=-1)
    
    # Use the loaded model to predict the emotion
    emotion_prediction = model.predict(face_roi)
    emotion_index = np.argmax(emotion_prediction)
    emotion_label = emotion_labels[emotion_index]
    
    # Draw a rectangle around the face and display the emotion label
    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    cv2.putText(img, emotion_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

# Display the image with emotion detection
cv2.imshow('Emotion Detection', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step


In [ ]:
import cv2
import numpy as np

# Load the Haar Cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the emotion recognition model (using OpenCV's SVM implementation)
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
fishface = cv2.ml.FisherFaceRecognizer_create()
fishface.read('mini_xception_model.h5')

# Function to extract features from the face region
def extract_features(gray, roi):
    roi_gray = gray[roi[1]:roi[1]+roi[3], roi[0]:roi[0]+roi[2]]
    roi_gray = cv2.resize(roi_gray, (48, 48))
    roi_gray = roi_gray.flatten()
    return roi_gray

# Open the video capture or load an image
cap = cv2.VideoCapture(0)  # Use 0 for webcam or provide a video file path
while True:
    # Read the frame
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Iterate over the detected faces
    for (x, y, w, h) in faces:
        # Extract features from the face region
        features = extract_features(gray, (x, y, w, h))

        # Predict the emotion using the SVM model
        predicted_emotion = fishface.predict(np.array([features]))[0]
        emotion_label = emotions[predicted_emotion]

        # Draw a rectangle around the face and display the emotion label
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, emotion_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

    # Display the resulting frame
    cv2.imshow('Emotion Detection', frame)

    # Exit the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()